# 6-5 Existence check of rows satisfying the conditions

## Customers who canceled thier reservation in 7 days

### Pandas

In [1]:
import pandas as pd

In [2]:
pd_reservation = pd.read_parquet("../../data/reservation.parquet")

In [3]:
pd_reservation.head()

,reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
0,1,2460,53431,2013-12-31 07:00:14+09:00,2014-12-31 00:00:00+09:00,2015-01-03 00:00:00+09:00,3,2,37800,reserved,NaT
1,2,962,488390,2013-12-31 08:23:35+09:00,2014-12-31 00:00:00+09:00,2015-01-02 00:00:00+09:00,2,3,42000,reserved,NaT
2,3,558,341335,2013-12-31 09:02:05+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,2,20400,reserved,NaT
3,4,3666,398981,2013-12-31 23:44:54+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,4,39600,reserved,NaT
4,5,2180,220381,2014-01-01 02:47:50+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,3,16500,reserved,NaT


In [4]:
(
    pd_reservation
    .assign(is_canceled_within_7days_to_checkin=lambda df:
            (df.status == "canceled") & ((df.checkin_date - df.canceled_at).dt.days <= 7))
    .groupby("customer_id")
    .is_canceled_within_7days_to_checkin
    .max()
)

customer_id
1         False
2         False
3         False
4         False
5         False
          ...  
499996    False
499997    False
499998     True
499999    False
500000    False
Name: is_canceled_within_7days_to_checkin, Length: 417368, dtype: bool

### Polars

In [5]:
import polars as pl

In [6]:
pl_reservation = pl.read_parquet("../../data/reservation.parquet")

In [7]:
pl_reservation.head()

reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
i64,i64,i64,"datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]",i64,i64,i64,str,"datetime[ns, Asia/Tokyo]"
1,2460,53431,2013-12-31 07:00:14 JST,2014-12-31 00:00:00 JST,2015-01-03 00:00:00 JST,3,2,37800,"""reserved""",null
2,962,488390,2013-12-31 08:23:35 JST,2014-12-31 00:00:00 JST,2015-01-02 00:00:00 JST,2,3,42000,"""reserved""",null
3,558,341335,2013-12-31 09:02:05 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,2,20400,"""reserved""",null
4,3666,398981,2013-12-31 23:44:54 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,4,39600,"""reserved""",null
5,2180,220381,2014-01-01 02:47:50 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,3,16500,"""reserved""",null


In [8]:
(
    pl_reservation
    .group_by("customer_id")
    .agg(
        (
            (pl.col("status") == "canceled") &
            ((pl.col("checkin_date") - pl.col("canceled_at")).dt.total_days() <= 7)
        ).max()
    ).sort("customer_id")
)

customer_id,status
i64,bool
1,false
2,false
3,false
4,false
5,false
…,…
499996,false
499997,false
499998,true
